In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.linalg import cholesky

def fit_mvn(data):
    """
    Fit a multivariate normal distribution to the data
    Returns: mean vector and covariance matrix
    """
    mean = np.mean(data, axis=0)
    cov = np.cov(data, rowvar=False)
    return mean, cov

def conditional_distribution_method1(mean, cov, x1_value):
    """
    Calculate conditional distribution using direct method
    Returns: conditional mean and variance
    """
    mu1, mu2 = mean
    sigma11 = cov[0,0]
    sigma12 = cov[0,1]
    sigma22 = cov[1,1]
    
    cond_mean = mu2 + (sigma12/sigma11) * (x1_value - mu1)
    cond_var = sigma22 - (sigma12**2)/sigma11
    
    return cond_mean, cond_var

def conditional_distribution_method2(mean, cov, x1_value):
    """
    Calculate conditional distribution using precision matrix method
    Returns: conditional mean and variance
    """
    precision = np.linalg.inv(cov)
    Lambda11 = precision[0,0]
    Lambda12 = precision[0,1]
    Lambda22 = precision[1,1]
    
    cond_var = 1/Lambda22
    cond_mean = mean[1] - (Lambda12/Lambda22) * (x1_value - mean[0])
    
    return cond_mean, cond_var

def simulate_conditional(mean, cov, x1_value, n_samples=100000):
    """
    Simulate from conditional distribution using Cholesky decomposition
    """
    # Get Cholesky decomposition
    L = cholesky(cov, lower=True)
    
    # Generate standard normal samples
    Z = np.random.standard_normal((n_samples, 2))
    
    # Transform to correlated normal
    X = Z @ L.T + mean
    
    # Adjust tolerance based on the scale of the data
    tolerance = np.std(X[:, 0]) * 0.1  # Use 10% of standard deviation as tolerance
    mask = np.abs(X[:, 0] - x1_value) < tolerance
    
    conditional_samples = X[mask, 1]
    
    # Ensure we have enough samples
    if len(conditional_samples) < 100:
        # If not enough samples, increase tolerance and try again
        tolerance = tolerance * 2
        mask = np.abs(X[:, 0] - x1_value) < tolerance
        conditional_samples = X[mask, 1]
    
    return conditional_samples

def analyze_mvn_data(file_path='problem3.csv'):
    """
    Main analysis function
    """
    try:
        # Read data
        df = pd.read_csv(file_path)
        data = df.values
        
        # A. Fit multivariate normal
        mean, cov = fit_mvn(data)
        print("A. Fitted Multivariate Normal Parameters:")
        print("Mean vector:")
        print(np.round(mean, 6))
        print("\nCovariance matrix:")
        print(np.round(cov, 6))
        print("\n")
        
        # B. Calculate conditional distribution for X2|X1=0.6
        x1_value = 0.6
        
        # Method 1: Direct method
        cond_mean1, cond_var1 = conditional_distribution_method1(mean, cov, x1_value)
        print(f"B. Conditional Distribution of X2|X1={x1_value}")
        print("\nMethod 1 (Direct):")
        print(f"Conditional Mean: {cond_mean1:.6f}")
        print(f"Conditional Standard Deviation: {np.sqrt(cond_var1):.6f}")
        
        # Method 2: Precision matrix method
        cond_mean2, cond_var2 = conditional_distribution_method2(mean, cov, x1_value)
        print("\nMethod 2 (Precision Matrix):")
        print(f"Conditional Mean: {cond_mean2:.6f}")
        print(f"Conditional Standard Deviation: {np.sqrt(cond_var2):.6f}")
        print("\n")
        
        # C. Simulation using Cholesky decomposition
        conditional_samples = simulate_conditional(mean, cov, x1_value)
        
        print("C. Simulation Results:")
        print(f"Number of samples near X1={x1_value}: {len(conditional_samples)}")
        print(f"Simulated Mean: {np.mean(conditional_samples):.6f}")
        print(f"Simulated Standard Deviation: {np.std(conditional_samples):.6f}")
        
        # Only perform normality test if we have enough samples
        if len(conditional_samples) >= 8:
            _, p_value = stats.normaltest(conditional_samples)
            print(f"Normality test p-value: {p_value:.6f}")
        else:
            print("Warning: Not enough samples for normality test")
        
        # Additional verification
        print("\nVerification of Results:")
        print("Comparing Methods (should be very close):")
        print(f"Mean difference between methods: {abs(cond_mean1 - cond_mean2):.6f}")
        print(f"Variance difference between methods: {abs(cond_var1 - cond_var2):.6f}")
        print(f"Mean difference from simulation: {abs(cond_mean1 - np.mean(conditional_samples)):.6f}")
        print(f"Variance difference from simulation: {abs(cond_var1 - np.var(conditional_samples)):.6f}")
        
        return {
            'mean': mean,
            'cov': cov,
            'cond_mean1': cond_mean1,
            'cond_var1': cond_var1,
            'cond_mean2': cond_mean2,
            'cond_var2': cond_var2,
            'conditional_samples': conditional_samples
        }
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    results = analyze_mvn_data()

A. Fitted Multivariate Normal Parameters:
Mean vector:
[0.046002 0.099915]

Covariance matrix:
[[0.010162 0.004924]
 [0.004924 0.020284]]


B. Conditional Distribution of X2|X1=0.6

Method 1 (Direct):
Conditional Mean: 0.368325
Conditional Standard Deviation: 0.133787

Method 2 (Precision Matrix):
Conditional Mean: 0.368325
Conditional Standard Deviation: 0.133787


C. Simulation Results:
Number of samples near X1=0.6: 0
Simulated Mean: nan
Simulated Standard Deviation: nan

Verification of Results:
Comparing Methods (should be very close):
Mean difference between methods: 0.000000
Variance difference between methods: 0.000000
Mean difference from simulation: nan
Variance difference from simulation: nan


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divi